# SCRAPE

In [ ]:
# === Installation (kun første gang i Colab) ===
!pip -q install praw pandas tqdm python-dateutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
import re, time, gc
from datetime import datetime, timezone
from dateutil import parser as dateparser

import pandas as pd
from tqdm import tqdm
import praw


## I have hidden the block of code which connects to the Reddit API as it has personal information (Secrets).

In [ ]:
DATE_FROM = "2023-01-01"
DATE_TO   = "2025-11-01"

AFTER  = dateparser.parse(DATE_FROM).replace(tzinfo=timezone.utc).timestamp()
BEFORE = dateparser.parse(DATE_TO).replace(tzinfo=timezone.utc).timestamp()

SUBS = [
    "vegan",
    "vegetarian",
    "nutrition",
    "environment",
    "climate",
    "AskUK",
    "AskEurope",
    "europe",
    "Futurology",
    "science",
    "CasualUK",
    "ukpolitics",
]

SUBMISSIONS_PER_SUB = 800
MAX_COMMENTS_TOTAL  = 1500
MIN_COMMENT_WORDS   = 5
OUT_CSV             = "reddit_comments_raw.csv"

DIET_RX  = re.compile(r'\b(vegan|vegetarian|plant[- ]?based|meatless|flexitarian|tofu|seitan|meat|beef|pork|chicken|bacon)\b', re.I)
SUST_RX  = re.compile(r'\b(sustainab\w*|climate|nitrogen|emission\w*|footprint|environment|CO2|carbon)\b', re.I)

PLANT_RX = re.compile(r'\b(vegan|vegetarian|plant[- ]?based|meatless|flexitarian|tofu|seitan)\b', re.I)
MEAT_RX  = re.compile(r'\b(meat|beef|pork|chicken|bacon|steak|lamb)\b', re.I)


def label_topic(text: str) -> str:
    if not isinstance(text, str):
        return "Other"
    has_plant = bool(PLANT_RX.search(text))
    has_meat  = bool(MEAT_RX.search(text))
    if has_plant and not has_meat:
        return "Plant-based"
    if has_meat and not has_plant:
        return "Meat"
    if has_plant and has_meat:
        return "Mixed"
    return "Other"


def englishish(body: str) -> bool:
    if not isinstance(body, str):
        return False
    b = body.strip()
    if not b:
        return False
    low = b.lower()
    if low in ("[deleted]", "[removed]"):
        return False
    if len(b.split()) < MIN_COMMENT_WORDS:
        return False
    ascii_ratio = sum(1 for ch in b if ord(ch) < 128) / max(1, len(b))
    return ascii_ratio > 0.7


rows = []
total = 0

for sub in SUBS:
    print(f"\n🔎 Subreddit: r/{sub}")
    subreddit = reddit.subreddit(sub)
    submissions = list(subreddit.new(limit=SUBMISSIONS_PER_SUB))

    for s in tqdm(submissions, desc=f"Scanning r/{sub}"):
        s_created = float(s.created_utc)
        if not (AFTER <= s_created <= BEFORE):
            continue
        try:
            post_text = (s.title or "") + "\n\n" + (s.selftext or "")
        except Exception:
            post_text = s.title or ""
        if not SUST_RX.search(post_text):
            continue
        try:
            s.comments.replace_more(limit=0)
        except Exception:
            time.sleep(0.5)
            continue
        for c in s.comments.list():
            try:
                body = c.body
                c_created = float(c.created_utc)
            except Exception:
                continue

            if not (AFTER <= c_created <= BEFORE):
                continue
            if not englishish(body):
                continue
            if not DIET_RX.search(body):
                continue

            rows.append({
                "id": c.id,
                "subreddit": str(c.subreddit),
                "created_utc": datetime.fromtimestamp(c_created, tz=timezone.utc).isoformat(),
                "body": body,
                "score": int(getattr(c, "score", 0)),
                "parent_id": getattr(c, "parent_id", None),
                "link_id": getattr(c, "link_id", None),
                "permalink": f"https://www.reddit.com{c.permalink}",
                "topic": label_topic(body),
                "post_title": s.title,
            })
            total += 1

            if total >= MAX_COMMENTS_TOTAL:
                break

        if total >= MAX_COMMENTS_TOTAL:
            break

    if total >= MAX_COMMENTS_TOTAL:
        break

df = pd.DataFrame(rows).drop_duplicates(subset=["id"]).reset_index(drop=True)
df.to_csv(OUT_CSV, index=False)
print(f"\n✅ Gemt: {OUT_CSV} | antal kommentarer: {len(df)}")
print("\nFordeling pr. topic:")
print(df["topic"].value_counts())



🔎 Subreddit: r/vegan


Scanning r/vegan: 100%|██████████| 800/800 [00:06<00:00, 124.83it/s]



🔎 Subreddit: r/vegetarian


Scanning r/vegetarian: 100%|██████████| 800/800 [00:03<00:00, 224.09it/s]



🔎 Subreddit: r/nutrition


...


🔎 Subreddit: r/CasualUK


Scanning r/CasualUK: 100%|██████████| 800/800 [00:00<00:00, 71657.69it/s]



🔎 Subreddit: r/ukpolitics


Scanning r/ukpolitics: 100%|██████████| 800/800 [00:00<00:00, 1127880.07it/s]


✅ Gemt: reddit_comments_raw.csv | antal kommentarer: 673

Fordeling pr. topic:
topic
Meat           310
Plant-based    246
Mixed          117
Name: count, dtype: int64


In [ ]:
pd.set_option("display.max_colwidth", None)

print("\n✨ Eksempler – Plant-based:")
display(df[df["topic"] == "Plant-based"].head(5)[["subreddit", "body", "score"]])

print("\n✨ Eksempler – Meat:")
display(df[df["topic"] == "Meat"].head(5)[["subreddit", "body", "score"]])



✨ Eksempler – Plant-based:


,subreddit,body,score
0,vegan,"\nThanks for posting to r/Vegan! &#128037;\n\nCivil discussion is welcome — personal attacks [are not](https://www.reddit.com/r/vegan/wiki/rules). Please [read our wiki](https://www.reddit.com/r/vegan/wiki/beginnersguide) first.\n\n**New to veganism?** &#127793; \n• Watch [Dominion](https://watchdominion.org/) — a powerful, free documentary that changes lives.\n• [NutritionFacts.org](https://NutritionFacts.org) — evidence-based health info \n• [HappyCow.net](https://HappyCow.net) — find vegan-friendly restaurants near you \n\n**Want to help animals?** &#128187; \n• Browse volunteer opportunities on [Flockwork](https://flockwork.org) and use your skills to make a difference \n• Join the [Flockwork Discord](https://discord.gg/4zmS2ADz3B) to be notified of new opportunities that match your skills\n\n**Thank you!**\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/vegan) if you have any questions or concerns.*",1
1,vegan,"If noone talks about it and gets challenged, the animals will stay exploited, enslaved and get murdered. \n\nTo people saying it's not black and white: Yes it is. Either the animal is left alone (vegan) or gets stabbed in the throat (not vegan).",48
2,vegan,"I just don't think that we ""force our views"" on others, at least not to the extent that we are made out to. You can just be minding your own business and once people find you're vegan, they will just accuse you of this\n\nNonvegans are always the ones being aggro from my experience",18
3,vegan,"Glad you agree we should talk about it, but vegans can't force you to be vegan. Non-vegans like you force animals into slaughterhouses. \n\nSo you are not in a position to give advice on activism, when you are still the reason we have to do activism. You are not vegan yet and still responsible for enslaving animals and abusing them as products. \n\nCheck out Dominion on YouTube[Dominion](https://youtu.be/LQRAfJyEsko?si=JVobQBW6RZc6f2hv)\n\nAnd if you don't know how easy it is to be vegan and leave animals alone, check out: how-vegan.com\n\nYou can do it 💪🏻",5
4,vegan,"Good that you want to change 💪🏻 So no animal has to be exploited for you anymore! \nIf you were in the victim's position -enslaved, trapped, misused as a product, before your murder - how fast would you want your oppressors to be vegan?",5



✨ Eksempler – Meat:


,subreddit,body,score
7,vegan,"For a tiny portion of your investments, look into Agronomics (ANIC) if you like the idea of cultured meat and precision fermentation. (this is a high risk stock, it can go up a lot but you're also likely to lose a lot, i.e. don't bet the farm)",2
13,vegan,">Basically, would you feel better eating meat if you knew that it was either obtained by the local farmer or your own (with farms being extremely small scale, possibly even just a chicken pen in your backyard), or hunted for in a far greater, stronger environment due to humans becoming far less spread out and pollution being cut be huge margins, by either you or your local hunter?\n\nI would feel a little less bad, but not enough to view eating meat as ethical.\n\nI also don't see why communism would be linked to increased quality of life for farmed animals - factory farming is the result of pressure to maximize output being more important than quality of life for the animals, and I doubt that replacing capitalism would replace the pressure to optimize. Historically, I don't think there's been an association between communism and farmed animal QOL (though that may not quite be a fair judgement given lot of communist societies were kind of strangled in the crib or co-opted by dictatorial regimes, but still - even in the more stable ones it didn't seem to happen).",1
16,vegan,"""how would your feelings change about meat, if at all, if we were in a communist society?""\n\nWhat a ridiculous question.\n\nMore humans have been killed, by far, in communist societies than anything else. \n\n""And also consider that in this hypothetical that there is no giant evil meat industry""\n\nI suggest your read a history book. Or even better, read The Gulag Archipelago by Solzhenitsyn. The book makes it crystal clear how communist ideology directly leads to mass scale atrocities. You will be ruled over by people, and they will see you as cattle. It kills individuality, personal freedom, and, ultimately, the human spirit. If one cares about facts and data at all, then there really is no debate.",-6
17,vegan,Interesting question! I think ethically sourced meat would feel way different in that scenario,-6
18,vegan,">factory farming is the result of pressure to maximize output being more important than quality of life for the animals\n\nDue to capitalist ideologies \n\n\n>and I doubt that replacing capitalism would replace the pressure to optimize.\n\nWell because things would be locally sourced and wouldn't require insane production due to people providing for themselves and their community, they wouldn't need ultra efficient farms because teaching people how to provide their own food would become far more important.\n\n\n>I would feel a little less bad, but not enough to view eating meat as ethical.\n\nBut this is still fair regardless of if it lowered optimisation",1


In [ ]:
import pandas as pd

df = pd.read_csv("reddit_comments_raw.csv")
pd.set_option("display.max_colwidth", None)

print("Number of comments:", len(df))
df.sample(10, random_state=42)[["subreddit", "body", "score", "topic"]]


Number of comments: 673


,subreddit,body,score,topic
212,vegan,Of course. Got to be the veganest vegan in veganreich.,-3,Plant-based
311,vegetarian,"The protein is such a huge factor for me. Yes, I enjoy milk in my cereal and coffee, but it's also a way for me to get more protein, especially when I'm not eating meat (I'm not vegetarian yet but I'm trying to get there). And personally I love the taste of soymilk but I get why other people might. It's just weird to think that I could buy a carton of soymilk and walk out holding more protein in that carton than there is on an entire shelf of oat milk.",14,Mixed
304,vegetarian,"they sell unsweetened soy at the local costco! in bulk! at a great price! it's shelf-stable before opening and organic! perfect for my oatmeal, smoothies, and home-brewed coffees. 😋 gonna try making tofu skins with it too.",2,Plant-based
299,vegetarian,"I forgot to mention - since you’re having a BBQ, you can cook these on the grill using a grill mat. Keep a mat for veggies and veggie burgers only. \n\nThe mats come in “copper” and black, so you can designate meat vs veggie.",4,Meat
631,science,No because the comparison is in no way related to which country is consuming more meat.\n\nIt’s just stating that US city emissions from meat consumption are comparable to all emissions from the UK - it’s just highlighting the scale.,27,Meat
660,science,Do you realize your B12 in beef is fed as a supplement to the cows? Do you realize milk is supplemented with 10+ vitamins and minerals? I'm sure you didn't..,3,Meat
500,science,My doctor (hematologist) is telling me to eat more red meat to help tackle my iron deficiency anemia because supplements alone aren't raising my levels to ideal levels. So for someone like me eating red meat should benefit my health.,41,Meat
468,science,"Nah, give me that healthy red meat!",0,Meat
109,vegan,"Thanks for sharing. This is similar to my journey, except when I started wfpb I'd already been vegan curious and identified as an environmentalist my entire life. Also I'd spent most of my life alternating between vegetarian, pescatarian and a flexitarian (though the word flexitarian hadn't been invented yet). So I was already on board and with vegan philosophy from the beginning. I think I'm equal parts vegan for health, vegan for the environment and vegan for the animals. There seem to be a lot of judgemental vegans who don't get how you can have multiple motivations working together",3,Plant-based
602,science,nobody except all the startups trying to grow lab meat and companies like beyond meat,1,Meat


# THEORY TESTING

### Helpers

In [ ]:
# === Regex-mønstre til TPB-komponenter ===
ATTITUDE_RX = re.compile(
    r"\b(i think|i believe|i feel|in my opinion|imo|i reckon|i guess|"
    r"good|bad|terrible|awful|amazing|great|disgusting|gross|"
    r"necessary|pointless|stupid|dumb|essential|important|"
    r"love|hate|prefer|like|dislike)\b",
    re.I
)

NORM_RX = re.compile(
    r"\b(people|everyone|everybody|most people|society|social|"
    r"friends|my friends|my family|family|parents|partner|boyfriend|girlfriend|"
    r"normal|normalised|accepted|expect|expectation|pressure|"
    r"real men|manly|masculine|men eat meat|vegan people|vegans|omnivores)\b",
    re.I
)

BARRIER_COST_RX = re.compile(
    r"\b(expensive|too expensive|costs? too much|cost of|afford|can'?t afford|"
    r"pricey|cheaper|more expensive)\b",
    re.I
)

BARRIER_CONV_RX = re.compile(
    r"\b(time|no time|takes too long|too long to cook|effort|too much effort|"
    r"inconvenient|hassle|hard to|difficult to|complicated|"
    r"no options|no option|lack of options|no choices|no choice|"
    r"available|availability|nowhere to buy|nowhere to get)\b",
    re.I
)

BARRIER_TASTE_RX = re.compile(
    r"\b(tastes? bad|taste bad|tastes? like shit|doesn'?t taste good|"
    r"flavourless|bland|disgusting|gross|i like the taste of meat|"
    r"craving|crave|i could never give up|addicted)\b",
    re.I
)

BARRIER_SOCIAL_RX = re.compile(
    r"\b(judged|judging|judge me|my friends would|my family would|"
    r"socially|awkward|embarrassing|embarrassed|"
    r"weird|seen as|look at me like|"
    r"political statement|virtue signalling|virtue-signalling|virtue signal)\b",
    re.I
)


In [ ]:
# Hjælpefunktioner til at matche mønstre
def has_pattern(rx, text):
    if not isinstance(text, str):
        return False
    return bool(rx.search(text))


df["has_attitude_text"] = df["body"].apply(lambda x: has_pattern(ATTITUDE_RX, x))
df["has_attitude"] = df["has_attitude_text"] | (df["compound"].abs() >= 0.3)

df["has_norm"] = df["body"].apply(lambda x: has_pattern(NORM_RX, x))

df["barrier_cost"] = df["body"].apply(lambda x: has_pattern(BARRIER_COST_RX, x))
df["barrier_convenience"] = df["body"].apply(lambda x: has_pattern(BARRIER_CONV_RX, x))
df["barrier_taste"] = df["body"].apply(lambda x: has_pattern(BARRIER_TASTE_RX, x))
df["barrier_social"] = df["body"].apply(lambda x: has_pattern(BARRIER_SOCIAL_RX, x))

df["has_barrier"] = df[[
    "barrier_cost",
    "barrier_convenience",
    "barrier_taste",
    "barrier_social"
]].any(axis=1)

df[[
    "has_attitude","has_norm","has_barrier",
    "barrier_cost","barrier_convenience","barrier_taste","barrier_social"
]].sum()


,0
has_attitude,547
has_norm,226
has_barrier,138
barrier_cost,47
barrier_convenience,87
barrier_taste,6
barrier_social,15


In [ ]:
def main_tpb_component(row):
    if row["has_barrier"]:
        return "Barrier-dominated (PBC)"
    if row["has_norm"]:
        return "Norm-dominated (Subjective norms)"
    if row["has_attitude"]:
        return "Attitude-dominated"
    return "Unclassified"

df["tpb_component"] = df.apply(main_tpb_component, axis=1)

df["tpb_component"].value_counts()


,count
tpb_component,
Attitude-dominated,288
Norm-dominated (Subjective norms),158
Barrier-dominated (PBC),138
Unclassified,89


## Attitude


In [ ]:
pd.set_option("display.max_colwidth", None)

print("\n=== Attitude-dominated ===")
display(df[df["tpb_component"]=="Attitude-dominated"]
        .sample(min(5, (df["tpb_component"]=="Attitude-dominated").sum()))
        [["body","sentiment","topic"]])


=== Attitude-dominated ===


,body,sentiment,topic
509,"Hello ive found success with textured soy protein, in particular textured soy slices. You just boil them, season them like meat (or marinate them) and then fry them or grill them. It scratches the meat itch i have been having, mainly if you use smoked pepper seasoning, liquid smoke or smoked salt to season it. And soy sauce, nutritional yeast, garlic, herbs. Then pan fry them or grill them or bake them. \n\nI eat them with side dishes like baked potatoes, toasted bread and dipping sauces.",positive,Meat
672,I wonder how much of this is aggravated by the price of beef,negative,Meat
117,"There is a better comparison in the article\n\n>More than 11 million tons of meat is consumed in U.S. cities annually -- equating to about 329 million tons of carbon emissions, according to a study published Monday in the scientific journal Nature Climate Change.\n\n>That figure is comparable to emission levels from domestic fossil fuel combustion in the U.S., at about 334 million tons annually,",positive,Meat
639,"Male milk cows are always killed and send to the slaughterhouse, or just tossed into the trash, since they cannot produce milk.\n\nFor female cows to constantly produce milk, they have to be artificially inseminated multiple times per year, by forcefully sticking a long metal rod into their vaginas and pumping semen into it.\n\nUntil the female cows are too old to produce the same amount of milk anymore, then they are slaughtered for meat or tossed into the trash.",negative,Meat
286,"Sweet Potato Veggie Burgers (Vegan + Gluten Free)\n\nMakes 10 small patties\n\n2 cups sweet potato, coarsely mashed\n\n1 can black beans (15oz)\n\n1 cup cooked brown rice\n\n1/2 of a red onion, chopped\n\n1 cup old fashioned oats (I used gluten free), ground\n\n1/2 cup loosely packed fresh cilantro, chopped\n\n1 large clove garlic\n\n1 tsp cumin\n\n1 tsp paprika\n\n1/2 tsp chili powder\n\nSalt and pepper to taste (probably used 1/2 tsp salt and a little less pepper)\n——-\n\n1. ﻿﻿﻿Cook the rice according to the package (bring water to a boil, add rice, simmer 35-40 mins); roughly 1/2 cup dry with make enough for the 1 cup of cooked rice you need.\n\n2. ﻿﻿﻿While the rice is cooking, peel your potato/s (one huge potato or two medium sized will probably be enough) and chop into small pieces. Toss them lightly in extra virgin olive oil before spreading on a baking sheet and placing in the oven for 20 minutes at 425 degrees Fahrenheit, flipping halfway through.When they're done, use a fork to coarsely mash them up (you want them to be chunky; not like a puree).\n\n3. Grind the oats into a flour using your food processor. Next add the beans, rice and onion into the food processor and pulse a few times. You want them to retain some structure and ""chunk"" You can add the garlic and cilantro in the food processor with the previous ingredients.\n\n4. In a large bowl, mix together the food processor contents, the sweet potato and all the remaining spices. I found it was most efficient to use my hands to do this.\n\n5. ﻿﻿﻿If the potatoes were still hot when you mixed everything together, you may want to stick the mixture in the refrigerator for 20 minutes.Otherwise, get right to the patty-making. Grabbing a small handful of the mixture, roll it into a ball in your hands and then gently press it into a patty shape.\n\n6. ﻿﻿﻿To serve, heat a splash of olive oil on medium-high heat on the stove top and cook the veggie burger for 3-4 minutes, flipping midway.\n\n7. ﻿﻿To store, wrap in a piece of parchment paper (so they don't stick to each other), place into a plastic Ziploc or Tupperware container, and freeze.﻿",positive,Plant-based


## Subjective norms

In [ ]:
print("\n=== Norm-dominated ===")
display(df[df["tpb_component"]=="Norm-dominated (Subjective norms)"]
        .sample(min(5, (df["tpb_component"]=="Norm-dominated (Subjective norms)").sum()))
        [["body","sentiment","topic"]])


=== Norm-dominated ===


,body,sentiment,topic
126,"Ok, go ahead and give up your phone, electricity, car, and every other modern convenience you have, as well as most of the other manufactured goods you consume on a daily basis. Go and convince everyone else to do the same.\n\nI prefer to operate in easy mode and stay vegan.",positive,Plant-based
643,"I absolutely agree that the ultra-rich should be held responsible, but it isn't enough, it must be a combination. The current lifestyle of normal people in Western countries is not sustainable by a wide margin.\n\nAnimal products cause approximately 16% of global GHG emissions. I'm not advocating that everybody should become vegan, I'm advocating for a sensible reduction of meat consumption to twice a week or so. Even if people would start with one vegetarian day per week I would be thrilled. Frankly, I don't think this is a big personal sacrifice.",positive,Mixed
537,"Everyone please check the methodology this is just junk science.\nHere we combine supply chain models with spatial carbon accounting to quantify and map the GHG emissions from beef, chicken and pork consumption\n""\n\n\nWhat their spatial carbon accounting is, well its a math equation where they guess how many cow farts they are. Theres no underlying data in this study.\n\n\nGigo. Garbage in garbage out.",positive,Meat
507,"Typically chicken and fish, but some people may go to tofu or plant-based meat alternatives, or simply eat less meat or meat-like foods in general.",neutral,Mixed
305,"maybe it’s just because of my job but I work at Starbucks so I deal with milk every day. oat milk is honestly the most common milk ordered in drinks behind whole milk or our standard (two percent). hell, even my partner will only drink oat milk unless I can’t get any then… he’ll use my milk; which is soy. oat milk is huge and it’s definitely dominating in the plant based milk market. I do remember growing up when it was almond milk but almond milk is a lot less commonly ordered; albeit it has to do with our almond milk being kinda dog water. I’ve been a soy milk drinker since infancy and I love, love soy. vanilla soy is my favorite but I tend to drink unsweetened soy milk now. oat milk isn’t bad but it’s a treat for me. oat milk raises your A1C and I’m inching towards pre diabetes currently. other than that? I see no appeal unless I wanted creamier oatmeal with a uniform taste.",positive,Plant-based


## Perceived behavioural control and barriers

In [ ]:
print("\n=== Barrier-dominated (PBC) ===")
display(df[df["tpb_component"]=="Barrier-dominated (PBC)"]
        .sample(min(5, (df["tpb_component"]=="Barrier-dominated (PBC)").sum()))
        [["body","sentiment","topic",
          "barrier_cost","barrier_convenience","barrier_taste","barrier_social"]])


=== Barrier-dominated (PBC) ===


,body,sentiment,topic,barrier_cost,barrier_convenience,barrier_taste,barrier_social
449,Why not invest heavy in campaigns and events for health died and information about sugar and red meat side effects instead on making it a privilege for those who can afford?,negative,Meat,True,False,False,False
428,"I always look to China for future things.\n\nBecause if you don't step away from the capitalist trend, your country is bound to look a lot like China or Hong Kong.\n\nEverything is efficient or automated and need little human interaction, but you'll live in smaller apartments, where 10sqm per person that come with a private toilet would be a luxury. Fortunately, every household item will be ""minimalist"" by default, so you won't need that much space...but you will really hate the way the walls feel like they're closing in on you every night.\n\nFood will be much cheaper and agriculture would be hyperfocused on eco-friendly ways and plants, but lab-made because it's just cheaper to mass produce lab-meats than to grow an entire cow.\n\n[“The New Protein Food Science and Technology Innovation Base will help complete the transformation of laboratory results into engineering and industrialisation, and lay a good development prospect for the commercialisation of cell-cultured meat,” Cui Xulong, Fengati District’s deputy mayor, said at the opening ceremony.](https://www.greenqueen.com.hk/china-lab-grown-meat-sustainable-food-protein-centre-technology/)\n\nWe'll probably have a few years eating insect protein first, though.\n\nThe usual beef, chicken, fish, and pork would be the future ""organic"" food. ""Not lab made"" would be the new marketing term.\n\nClothes will be almost all plastic because they're more durable.\n\nI can go on and on, but generally, the world will be hyperefficient and doing things manually would be what rich people do.\n\nCan't comment too far about jobs, but the sex industry will be heavily reliant on them. Just not sure if it will curb the number of real prostitutes/pornstars or actually drive demand because AI porn would be considered cheap or ""for the masses,"" and watching real people do it would be premium.",positive,Meat,True,False,False,False
95,"i first went vegetarian at 16 (am 35 now), mostly because my older sister made me have an absolute fit after explaining what sausage actually was. i knew meat was animals but the vivid detail really did a number on me mentally. so it was very easy to just not eat meat from then on. but as it was still he mid 2000s then and i was a minor i didn't have the accessibility to do more than that. it wasn't until i was in my 20s and more food options became available that i slowly stopped eating eggs and dairy in things completely as i swapped for the new available alternatives.\n\ntbh i am surprised how relatively easy it is even living in the 'bbq belt' though eating out can be a struggle so i just don't do it much at all. or if i have to go to a family dinner at a restaurant i just eat before or make do with what's there. luckily my family is pretty supportive if not more like 'neutral' about it? i've had my sisters and a cousin's bf who is a chef make things they know i can eat since we do potluck for most functions/gatherings at each others' houses and always have even well before any diet changes. i guess i can call myself lucky. nothing really was hard per se and these days it's fun to try new recipes. food science is fun.",positive,Mixed,False,True,False,False
303,"Soy milk is still the cheapest option in France and I use it sometimes. It remains widely available so it's cool. I do prefer the less popular rice milk for taste though. As for what baristas offer in cafes and such.... Well, it's a mix between soy and oat. I'm gluten intolerant so I don't trust oat milk (I assume it can be gluten free but the ones I see the most aren't). And if there is no soy option, I'll take regular cow milk tbh. France genuinely has a lot of options for plant based milk. Hazeln

In [ ]:
df.to_csv("reddit_comments_tpb_coded.csv", index=False)
print("✅ Gemt: reddit_comments_tpb_coded.csv")


✅ Gemt: reddit_comments_tpb_coded.csv


In [ ]:
from google.colab import files
files.download("reddit_comments_tpb_coded.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Classification

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/reddit_comments_cleaned.csv")
df.head()

,id,subreddit,created_utc,body,score,parent_id,link_id,permalink,topic,post_title,...,has_attitude_text,has_attitude,has_norm,barrier_cost,barrier_convenience,barrier_taste,barrier_social,has_barrier,tpb_component,irrelevant
0,nmgbdl9,vegan,2025-10-31T23:16:11+00:00,"If noone talks about it and gets challenged, t...",48,t3_1ol8n44,t3_1ol8n44,https://www.reddit.com/r/vegan/comments/1ol8n4...,Plant-based,On the topic of forcing your views on others,...,False,True,True,False,False,False,False,False,Norm-dominated (Subjective norms),False
1,nmgbn9k,vegan,2025-10-31T23:17:56+00:00,"I just don't think that we ""force our views"" o...",18,t3_1ol8n44,t3_1ol8n44,https://www.reddit.com/r/vegan/comments/1ol8n4...,Plant-based,On the topic of forcing your views on others,...,False,False,True,False,False,False,False,False,Norm-dominated (Subjective norms),False
2,nmgc1yy,vegan,2025-10-31T23:20:33+00:00,"Glad you agree we should talk about it, but ve...",5,t1_nmg8tjc,t3_1ol8n44,https://www.reddit.com/r/vegan/comments/1ol8n4...,Plant-based,On the topic of forcing your views on others,...,True,True,True,False,False,False,False,False,Norm-dominated (Subjective norms),False
3,nmge6od,vegan,2025-10-31T23:34:23+00:00,Good that you want to change 💪🏻 So no animal h...,5,t1_nmgcgh5,t3_1ol8n44,https://www.reddit.com/r/vegan/comments/1ol8n4...,Plant-based,On the topic of forcing your views on others,...,True,True,False,False,False,False,False,False,Attitude-dominated,False
4,nm9qod2,vegan,2025-10-30T22:05:01+00:00,"For a tiny portion of your investments, look i...",2,t3_1ok13sh,t3_1ok13sh,https://www.reddit.com/r/vegan/comments/1ok13s...,Meat,"Those who invest, what do you invest in?",...,True,True,False,False,False,False,False,False,Attitude-dominated,False


In [3]:
df_rel = df[df['irrelevant'] == 0].copy()

attitudes = df_rel[df_rel['tpb_component'] == 'Attitude-dominated'].copy()
norms     = df_rel[df_rel['tpb_component'] == 'Norm-dominated (Subjective norms)'].copy()
barriers  = df_rel[df_rel['tpb_component'] == 'Barrier-dominated (PBC)'].copy()

print("Attitude comments:", len(attitudes))
print("Norm comments:", len(norms))
print("Barrier/PBC comments:", len(barriers))



Attitude comments: 212
Norm comments: 141
Barrier/PBC comments: 114


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

def cluster_comments(texts, k=3):
    texts = pd.Series(texts).fillna("").astype(str)
    mask_nonempty = texts.str.strip() != ""
    texts_clean = texts[mask_nonempty]

    n = len(texts_clean)
    if n == 0:
        return pd.Series([pd.NA] * len(texts), index=texts.index)
    if n < k:
        k = n

    vec = TfidfVectorizer(stop_words="english", max_features=3000)
    X = vec.fit_transform(texts_clean)

    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = km.fit_predict(X)
    cluster_full = pd.Series(index=texts.index, dtype="float")
    cluster_full[mask_nonempty] = clusters

    return cluster_full


## Attitudes

In [7]:
attitudes['cluster'] = cluster_comments(attitudes['body'], k=3)


## Norms

In [8]:
norms['cluster'] = cluster_comments(norms['body'], k=3)

## Barriers

In [9]:
barriers['cluster'] = cluster_comments(barriers['body'], k=2)


## Clusters

### Attitudes

In [10]:
for c in sorted(attitudes['cluster'].dropna().unique()):
    print(f"\n=== Attitude cluster {int(c)} ===")
    for txt in attitudes[attitudes['cluster'] == c]['body'].head(100):
        print("-", txt[:250].replace("\n", " "))



=== Attitude cluster 0 ===
- Interesting question! I think ethically sourced meat would feel way different in that scenario
- David Attenborough isnt vegan though  He literally says in A Life on Our Planet "I rarely eat meat"   In an recent interview he said  "I have certainly changed my diet. Not in a great sort of dramatic way. **But I don’t think I’ve eaten red meat for 
- It's a fair concern. But if the meat industry has to compete with lab grown meat it could have a great impact!
- Does he seriously think he’s going to sell meat alternatives to the MAGA crowd? What a fool
- The original marketing was that it bleeds like real meat, which is about his anti-woke as you can get.    PLEASE don’t tell me my favorite fake meat has sold out to some right wing @ss-f4ck!  I’ve had dreams of working for that company
- Vegetarian versions of any regular meat or fish. I would ask which kinds and brands they they prefer. They probably have their favourite vegetarian sausages and burgers, etc.


### Norms

In [11]:
for c in sorted(norms['cluster'].dropna().unique()):
    print(f"\n=== Norms cluster {int(c)} ===")
    for txt in norms[norms['cluster'] == c]['body'].head(300):
        print("-", txt[:250].replace("\n", " "))



=== Norms cluster 0 ===
- I just don't think that we "force our views" on others, at least not to the extent that we are made out to. You can just be minding your own business and once people find you're vegan, they will just accuse you of this  Nonvegans are always the ones 
- Lots of people hate leather, especially in couches or car seats.  Many years ago, I wrote to BMW to complain about their skin seats, seats which did not match their concept of "Joy".  As a result, the following spring they offered non-leather seats a
- >  Heck a lot of "vegans" will buy second hand leather.   I don't see the need to put vegans in quotes here. Someone can absolutely be vegan and buy second hand leather. It's a definite grey area and IMO not worth focusing our attention on.
- Yeah I think the quotation marks suggest I'm more opinionated about it than I am.  I do see that it is a bit of a grey area and can understand both sides. I more meant the quotes as in people will say they are vegan and stil

## General

In [ ]:
# N-count
attitudes['cluster'].value_counts()


,count
cluster,
1.0,99
0.0,59
2.0,54


In [ ]:
norms['cluster'].value_counts()

,count
cluster,
2.0,61
0.0,42
1.0,38


# PBC and barriers

In [12]:
cost_comments = barriers[barriers["barrier_cost"] == 1]
convenience_comments = barriers[barriers["barrier_convenience"] == 1]
taste_comments = barriers[barriers["barrier_taste"] == 1]
social_comments = barriers[barriers["barrier_social"] == 1]


In [13]:
barriers[["barrier_cost","barrier_convenience","barrier_taste","barrier_social"]].sum()


,0
barrier_cost,40
barrier_convenience,69
barrier_taste,6
barrier_social,15


In [14]:
cost_comments = barriers[barriers["barrier_cost"] == 1]["body"]
convenience_comments = barriers[barriers["barrier_convenience"] == 1]["body"]
taste_comments = barriers[barriers["barrier_taste"] == 1]["body"]
social_comments = barriers[barriers["barrier_social"] == 1]["body"]


In [15]:
def show_barrier_comments(df, barrier_column, limit=None):
    subset = df.loc[df[barrier_column] == 1, "body"]

    print(f"\n=== {barrier_column.upper()} (n={len(subset)}) ===")

    for i, txt in enumerate(subset):
        if limit is not None and i >= limit:
            break
        print("-", str(txt).strip().replace("\n", " "))


In [16]:
show_barrier_comments(barriers, "barrier_cost")
show_barrier_comments(barriers, "barrier_convenience")
show_barrier_comments(barriers, "barrier_taste")
show_barrier_comments(barriers, "barrier_social")



=== BARRIER_COST (n=40) ===
- Most vegans oppose cruelty to animals no matter how small, family-oriented, local, traditional, or humane-washed it is. The size of animal agriculture is a symptom, but it's not the big picture. The fact is that on almost every level, the production and consumption of animal product is wholly unnecessary. The vast majority of people on earth live in areas where plant based foods are cheaper, more abundant, and more sustainable. We can slap leftist ideologies onto animal abuse, but it won't suddenly make it magically acceptable as a result. If anything, progressive politics and veganism should co-exist perfectly well together. Especially at a feminist level, given that animal agriculture is known for exploiting the magic of the female body for ultra capitalist gain.
- I always thought of leather as a bi product of the meat industry. So the animal will be killed for meat regardless. Therefore the meat industry will just have to sell the skin on for leather 